# Matching trains with traffic lines

The main goal here is to develop a function that makes it possible to identify the line of a specific delayed train. The reason why we need such a function is because the passenger ridership estimation is given per line where the delay data is per specific train.

## Importing datasets

First, let us import the dataset for all the traffic lines (used in the ridership estimation data).

In [58]:
# import excel file static_pass_all_2024.xlsx

import pandas as pd
import os


# read by default 1st sheet of an excel file
df_line = pd.read_excel('static_pass_all_2024.xlsx')

In [59]:
# drop all the columns except the first 3 (no need for ridership data, only the line number, name and stopping patterns are of interest)
df_line = df_line.iloc[:, :9]

Let us now import the train data, more specifically the trains that are affected by delays. Of interest here are particularly Tågnr	and Tåguppdrag.
The goal is to match all of them to a specific line number in df_line.

In [60]:
# read by default 1st sheet of an excel file
df_train = pd.read_excel('metatraindata_2023.xlsx')

Let us also read the Lupp data where we have more attributes for each train, more particularly the stopping pattern. There are four different files in the data folder named as follows Rapport_T23_vX.csv where X is 11, 19, 28 and 37, we will read all of these and combine them in one dataframe, note the first row of each file is the header.

In [61]:
import pandas as pd
import glob

# Define the folder path and file pattern
folder_path = 'data/'  # Adjust the folder path if needed
file_pattern = 'Rapport_T23_v*.csv'

# Use glob to find all matching files
file_paths = glob.glob(folder_path + file_pattern)

# Read all files into a list of DataFrames
dfs = [pd.read_csv(file, header=0) for file in file_paths]

# Combine all DataFrames into one
df_lupp = pd.concat(dfs, ignore_index=True)

We need to clean up (make this df a bit smaller), e.g., by removing unnecessary data.

In [62]:
# from combined_df remove the following columns
# År (PAU)
# Veckonr (PAU)
# Datum (PAU)
# Tågslag, but before remove all raws where Tågslag is not RST

df_lupp_rst = df_lupp[df_lupp['Tågslag'] == 'RST']
df_lupp_rst_clean = df_lupp_rst.drop(columns=['År (PAU)', 'Veckonr (PAU)', 'Datum (PAU)', 'Tågslag'])

In [63]:
# remove all rows where both Uppehållstypavgång is Passage and Uppehållstypankomst is Passage
df_lupp_rst_clean = df_lupp_rst_clean[(df_lupp_rst_clean['Uppehållstypavgång'] != 'Passage') | (df_lupp_rst_clean['Uppehållstypankomst'] != 'Passage')]

In [64]:
# check how many trains from df_train that are in df_lupp_rst_clean
# for that search using the column Tågnr and Tåguppdrag from df_train
# and use similar columns Tåguppdrag and Tågnr from df_lupp_rst_clean
# to find the matching trains

# make sure these are int in both dataframes
df_train['Tågnr'] = df_train['Tågnr'].astype('Int64')
df_train['Tåguppdrag'] = df_train['Tåguppdrag'].astype('Int64')
df_lupp_rst_clean['Tåguppdrag'] = df_lupp_rst_clean['Tåguppdrag'].astype('Int64')

# in df_lupp_rst_clean, remove spaces between numbers first in Tågnr
# Remove spaces between numbers in the Tågnr column
df_lupp_rst_clean['Tågnr'] = df_lupp_rst_clean['Tågnr'].astype(str).str.replace(r'\s+', '', regex=True)
df_lupp_rst_clean['Tågnr'] = df_lupp_rst_clean['Tågnr'].astype('Int64')

In [65]:
# for each Tågnr, print how many possible Tåguppdrag there are
# this is to see if there are any duplicates in the data
x = df_lupp_rst_clean.groupby('Tågnr')['Tåguppdrag'].nunique()
y = df_train.groupby('Tågnr')['Tåguppdrag'].nunique()
# print the max and min for each dataframe
print(x.max(), x.min())
print(y.max(), y.min())

1 1
3 1


## Matching train delay and Lupp data

Before trying to find the closest line (line number/name) to a certain train (tågnr/uppdrag). Let us first look att how many delayed trains can we identify in the sample of Lupp data that we have.

In [66]:
# Remove duplicates from df_train and combined_df based on ('Tågnr', 'Tåguppdrag')
df_train_test = df_train.drop_duplicates(subset=['Tågnr', 'Tåguppdrag'])
combined_df_test = df_lupp_rst_clean.drop_duplicates(subset=['Tågnr', 'Tåguppdrag'])

# Perform an inner merge to find matching trains
matching_trains = pd.merge(
    df_train_test, 
    combined_df_test, 
    how='inner', 
    left_on=['Tågnr', 'Tåguppdrag'], 
    right_on=['Tågnr', 'Tåguppdrag']
)

# Count the number of matching trains
num_matching_trains = matching_trains.shape[0]
print(f"Number of matching trains: {num_matching_trains}")

# Count the number of unique trains in df_train
num_unique_trains = len(df_train_test[['Tåguppdrag']])
print(f"Out of {num_unique_trains} unique trains")

# Calculate the percentage of matching trains
matching_percentage = num_matching_trains / num_unique_trains * 100
print(f"Percentage of matching trains: {matching_percentage:.2f}%")

Number of matching trains: 5833
Out of 14474 unique trains
Percentage of matching trains: 40.30%


We now know that we have stopping pattern information (from Lupp data T23) for around 40% of the delayed trains (in metatraindata_2023). From now on, we focus on matching these 40% delayed trains to their line numbers.

First, we append the stopping pattern information to our delayed trains.

In [67]:
filtered_stops = df_lupp_rst_clean[
    ((df_lupp_rst_clean['Uppehållstypavgång'].isin(['Uppehåll', 'Första']))) |
    ((df_lupp_rst_clean['Uppehållstypankomst'].isin(['Sista'])))
]

first_dates = filtered_stops.groupby(['Tågnr', 'Tåguppdrag'])['Datum'].min().reset_index()
filtered_stops = pd.merge(filtered_stops, first_dates, on=['Tågnr', 'Tåguppdrag', 'Datum'])

stops_per_train = (
    filtered_stops.groupby(['Tågnr', 'Tåguppdrag'], as_index=False)
    .agg({'Delsträckanummer': list, 'Avgångplatssignatur': list, 'Uppehållstypankomst': list, 'AnkomstplatsPlatssignatur': list})
    .apply(lambda x: pd.Series({
        'Tågnr': x['Tågnr'],
        'Tåguppdrag': x['Tåguppdrag'],
        'Stopps': (
            [stop for i, stop in zip(x['Delsträckanummer'], x['Avgångplatssignatur']) 
             if pd.notna(stop)] +
            [x['AnkomstplatsPlatssignatur'][i] for i, type_a in enumerate(x['Uppehållstypankomst']) 
             if type_a == 'Sista' and pd.notna(x['AnkomstplatsPlatssignatur'][i])]
        )
    }), axis=1)
)

train_stops = pd.merge(
    matching_trains, 
    stops_per_train, 
    how='inner', 
    on=['Tågnr', 'Tåguppdrag']
)[['Tågnr', 'Tåguppdrag', 'Stopps']]

# make sure all the stops are uppercase
train_stops['Stopps'] = train_stops['Stopps'].apply(lambda x: [stop.upper() for stop in x])

# some trains have the same stops, so remove duplicates
train_stops_no_duplicates = train_stops.drop_duplicates(subset=['Stopps'])

In [68]:
# Group by 'Linje' and combine the 'från_sign' and 'till_sign' for each line
line_stops = df_line.groupby('Linje').apply(
    lambda x: list(x['från_sign']) + [x['till_sign'].iloc[-1]]
).reset_index()

# Rename columns for clarity
line_stops.columns = ['Linje', 'Stopps']

C:\Users\AbdouAA\AppData\Local\Temp\ipykernel_17028\3534091136.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  line_stops = df_line.groupby('Linje').apply(


## Matching delayed trains to traffic lines

We now match delayed trains (subset with unique stop patterns) to the most likely traffic line. The most likely line is chosen as the one with the highest similarity score.

In [69]:
import pandas as pd
from difflib import SequenceMatcher

def calculate_score(train_stops, line_stops):
    """
    Calculate a similarity score between train stops and line stops.
    """
    # Match first and last stop
    score = 0
    if train_stops[0] == line_stops[0]:
        score += 2  # Higher weight for matching first stop
    if train_stops[-1] == line_stops[-1]:
        score += 2  # Higher weight for matching last stop
    
    # Calculate sequence similarity for intermediate stops
    sequence_similarity = SequenceMatcher(None, train_stops, line_stops).ratio()
    score += sequence_similarity * 10  # Adjust weight for sequence similarity
    
    return score

def get_inverted_line(line_id):
    """
    Get the inverted line ID.
    """
    return line_id[:-1] if line_id.endswith('R') else f"{line_id}R"

def match_trains_to_lines(train_stops_df, line_stops_df):
    """
    Match trains to lines based on similarity scores, including inverted stops.
    """
    matches = []
    for _, train_row in train_stops_df.iterrows():
        best_score = -1
        best_match = None
        best_direction = 'Normal'
        
        for _, line_row in line_stops_df.iterrows():
            # Calculate score for normal stops
            normal_score = calculate_score(train_row['Stopps'], line_row['Stopps'])
            
            # Calculate score for inverted stops
            inverted_stops = line_row['Stopps'][::-1]
            inverted_score = calculate_score(train_row['Stopps'], inverted_stops)
            
            # Determine better match (normal or inverted)
            if inverted_score > normal_score:
                current_score = inverted_score
                current_match = get_inverted_line(line_row['Linje'])
                current_direction = 'Inverted'
            else:
                current_score = normal_score
                current_match = line_row['Linje']
                current_direction = 'Normal'
            
            # Update best match
            if current_score > best_score:
                best_score = current_score
                best_match = current_match
                best_direction = current_direction
        
        matches.append({
            'Tågnr': train_row['Tågnr'],
            'Predicted_Line': best_match,
            'Score': best_score,
            'Direction': best_direction
        })
    
    return pd.DataFrame(matches)

# Example usage
train_stops_df = train_stops_no_duplicates
line_stops_df = line_stops

result = match_trains_to_lines(train_stops_df, line_stops_df)

# Add a column to results corresponding stops of the predicted line
result = pd.merge(result, line_stops, left_on='Predicted_Line', right_on='Linje', how='left').rename(columns={'Stopps': 'Stopps_line'})
result = pd.merge(result, train_stops_df, left_on='Tågnr', right_on='Tågnr', how='left')

Now, we can construct the final table where all the trains are identified with a specific traffic line.

In [70]:
# Using the matching found in the previous step, i.e., in result, we want to add a column to the train_stops which include the predicted line
# we have previously based the matching on the train stops with no duplicates train_stops_no_duplicates = train_stops.drop_duplicates(subset=['Stopps'])

# Convert lists to tuples to make them hashable
train_stops_no_duplicates['Stopps'] = train_stops_no_duplicates['Stopps'].apply(tuple)
result['Stopps'] = result['Stopps'].apply(tuple)
train_stops['Stopps'] = train_stops['Stopps'].apply(tuple)

# Merge train_stops with result based on 'Stopps' to add predicted line
train_stops_with_lines = pd.merge(
    train_stops, 
    train_stops_no_duplicates[['Stopps']].merge(
        result[['Stopps', 'Predicted_Line']],
        on='Stopps',
        how='left'
    ),
    on='Stopps',
    how='left'
)


C:\Users\AbdouAA\AppData\Local\Temp\ipykernel_17028\3374585376.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_stops_no_duplicates['Stopps'] = train_stops_no_duplicates['Stopps'].apply(tuple)


In [57]:
# save the results to an excel file
# * partial because the Lupp data that was used to identify the stops is only for some weeks in 2023, 40% of the delayed trains in 2023 have still been successfully identified!
train_stops_with_lines.to_excel('train_stops_with_predicted_lines_partial_2023.xlsx', index=False)